In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it", trust_remote_code=True)




tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [3]:
tokenizer.chat_template="""{{ bos_token }}
{%- if messages[0]['role'] == 'system' -%}
    {%- if messages[0]['content'] is string -%}
        {%- set first_user_prefix = messages[0]['content'] + '\n\n' -%}
    {%- else -%}
        {%- set first_user_prefix = messages[0]['content'][0]['text'] + '\n\n' -%}
    {%- endif -%}
    {%- set loop_messages = messages[1:] -%}
{%- else -%}
    {%- set first_user_prefix = "" -%}
    {%- set loop_messages = messages -%}
{%- endif -%}

{%- if not tools is defined -%}
    {%- set tools = none -%}
{%- endif -%}

{%- for message in loop_messages -%}
    {%- if (message['role'] == 'user') != (loop.index0 % 2 == 0) -%}
        {{ raise_exception("Conversation roles must alternate user/assistant/user/assistant/...") }}
    {%- endif -%}
    {%- if (message['role'] == 'assistant') -%}
        {%- set role = "model" -%}
    {%- else -%}
        {%- set role = message['role'] -%}
    {%- endif -%}
    
    {%- if message['role'] == 'tool' -%}
        {#- Handle tool results as user messages -#}
        {{ '<start_of_turn>user\n' }}
        {{- 'Tool Result: ' + message['content'] + '\n' }}
        {{ '<end_of_turn>\n' }}
    {%- else -%}
        {{ '<start_of_turn>' + role + '\n' + (first_user_prefix if loop.first else "") }}
        
        {%- if message['role'] == 'user' and tools is not none and loop.first -%}
            {#- Inject tools into first user message only -#}
            {%- if message['content'] is string -%}
                {%- set user_content = message['content'] | trim -%}
            {%- elif message['content'] is iterable -%}
                {%- set user_content = '' -%}
                {%- for item in message['content'] -%}
                    {%- if item['type'] == 'text' -%}
                        {%- set user_content = user_content + item['text'] | trim -%}
                    {%- endif -%}
                {%- endfor -%}
            {%- endif -%}
            
            {{- 'Available Tools: You can call functions using JSON format like {"name": "function_name", "arguments": {"param": "value"}}.\n\n' }}
            {%- for tool in tools -%}
                {{- tool.function.name + ': ' + tool.function.description + '\n' }}
                {{- 'Parameters: ' + (tool.function.parameters | tojson) + '\n\n' }}
            {%- endfor -%}
            {{- 'User Query: ' + user_content }}
            
        {%- elif message['role'] == 'assistant' and message.get('tool_calls') -%}
            {#- Handle assistant tool calls -#}
            {%- if message.get('content') -%}
                {{- message['content'] | trim + '\n\n' }}
            {%- endif -%}
            {%- for tool_call in message['tool_calls'] -%}
                {{- '{"name": "' + tool_call.function.name + '", "arguments": ' + tool_call.function.arguments + '}' }}
                {%- if not loop.last -%}{{ '\n' }}{%- endif -%}
            {%- endfor -%}
            
        {%- else -%}
            {#- Regular message handling (preserves original logic) -#}
            {%- if message['content'] is string -%}
                {{ message['content'] | trim }}
            {%- elif message['content'] is iterable -%}
                {%- for item in message['content'] -%}
                    {%- if item['type'] == 'image' -%}
                        {{ '<start_of_image>' }}
                    {%- elif item['type'] == 'text' -%}
                        {{ item['text'] | trim }}
                    {%- endif -%}
                {%- endfor -%}
            {%- else -%}
                {{ raise_exception("Invalid content type") }}
            {%- endif -%}
        {%- endif -%}
        
        {{ '<end_of_turn>\n' }}
    {%- endif -%}
{%- endfor -%}

{%- if add_generation_prompt -%}
    {{'<start_of_turn>model\n'}}
{%- endif -%}"""

In [5]:
tokenizer.apply_chat_template([{
  "messages": [
    {
      "role": "user",
      "content": "What's the weather in Paris and what is 123*456?"
    }
  ],
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["celsius", "fahrenheit"]
            }
          },
          "required": ["location"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "calculate",
        "description": "Do a math calculation",
        "parameters": {
          "type": "object",
          "properties": {
            "expression": {
              "type": "string",
              "description": "A math expression, e.g. 2+2*5"
            }
          },
          "required": ["expression"]
        }
      }
    }
  ],
  "tool_choice": "auto"
}]
)

TemplateError: Conversation roles must alternate user/assistant/user/assistant/...